In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import pymorphy2
import nltk
import stop_words
from nltk.corpus import stopwords

In [2]:
train = pd.read_csv("train.csv", sep='|')
test = pd.read_csv("test.csv", sep='|')

In [3]:
test[:10]

,index,reviewText
0,0,So many reviewers have already given you the p...
1,1,The film opens up with an ideal family. When t...
2,2,Why does everyone like this stupid movie so mu...
3,3,Fair watch. Unlike some shows it got better wi...
4,4,I thought I was buying the DVD of the Season 4...
5,5,"I turned it off when Artie is in the filthy, n..."
6,6,What television series has maintained quality ...
7,7,I cannot say anything about the quality of the...
8,8,When DeMornay goes off in a bathroom stall and...
9,9,Have been a fan since it started 5 years ago a...


In [4]:
train.overall.value_counts()

1    25000
0    15000
Name: overall, dtype: int64

In [5]:
test.head()

,index,reviewText
0,0,So many reviewers have already given you the p...
1,1,The film opens up with an ideal family. When t...
2,2,Why does everyone like this stupid movie so mu...
3,3,Fair watch. Unlike some shows it got better wi...
4,4,I thought I was buying the DVD of the Season 4...


In [6]:
train.isnull().sum()

overall       0
reviewText    0
dtype: int64

In [7]:
from sklearn.metrics import f1_score

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer

In [9]:
from nltk.stem.wordnet import WordNetLemmatizer

In [10]:
lmtzr = WordNetLemmatizer()

In [13]:
train.reviewText[:10]

0    Entertaining enough for those who don't think ...
1    I bought it yesterday havent started watching ...
2    This movie tells the story of three kids who g...
3    You wanna know what its like for a Black perso...
4    Warner Archive has finally released an epic fi...
5    Here is an excellent low-budget film produced ...
6    That part alone was not a redeeming factor for...
7    i love old westerns this is a great collection...
8    Warner Archive has once again done it.  They h...
9    I remember reading the phantom of the opera ba...
Name: reviewText, dtype: object

In [36]:
nltk.word_tokenize('Entertaining enough for those who dont think ')

['Entertaining', 'enough', ',', 'for', 'those', 'who', 'dont', 'think']

In [22]:
lmtzr.lemmatize('bought', 'v')

'buy'

In [43]:
vocabulary = set()
for index, sentence in train.reviewText[:10].iteritems():
    text = ''
    for word in nltk.word_tokenize(sentence):
        if word not in stopwords.words('english'):
            text += lmtzr.lemmatize(word,'v') + ' '
    train.loc[index, 'reviewText'] = text

In [44]:
train.reviewText[:10]

0    Entertaining enough n't think much.There , I s...
1    I buy yesterday havent start watch yet Im go ....
2    This movie tell story three kid grow slum area...
3    You wan na know like Black person kid ? Check ...
4    Warner Archive finally release epic film get s...
5    Here excellent low-budget film produce Canada ...
6    That part alone redeem factor film . I 'm go e...
7    love old westerns great collection take back c...
8    Warner Archive . They release 40 year old film...
9    I remember read phantom opera back school , fa...
Name: reviewText, dtype: object

In [ ]:
test_text_lem = []
for sentence in test.reviewText[]:
    test_text_lem.append([])
    for word in nltk.word_tokenize(sentence):
        if word not in stopwords.words('english'):
            test_text_lem[-1].append(lmtzr.lemmatize(word))

In [10]:
pipeline_lr = Pipeline([('vectorizer', CountVectorizer(binary=True,ngram_range=(1,3))),
                     ('clf_lr', LogisticRegression(C = 0.1))])

In [12]:
#pipeline_lr = Pipeline([('vect', CountVectorizer(binary=True,ngram_range=(1,3))), 
#                        ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))])

In [11]:
scores = cross_val_score(pipeline_lr, train.reviewText, train.overall, cv=9, n_jobs=-1, scoring="roc_auc")
#GridSearchCV

In [14]:
scores = cross_val_score(pipeline_lr,[" ".join(i) for i in train_text_lem], train.overall, cv=3, n_jobs=-1, scoring="roc_auc")

In [12]:
scores.mean()

0.95392069129295032

In [13]:
pipeline_lr.fit(train["reviewText"],train["overall"])

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [14]:
pipeline_lr.predict_proba(test["reviewText"])[:,1]

array([ 0.78364767,  0.49257716,  0.00261731, ...,  0.66000477,
        0.00729707,  0.31197752])

In [18]:
sorted(pipeline_lr.get_params().keys())

['clf_lr',
 'clf_lr__C',
 'clf_lr__class_weight',
 'clf_lr__dual',
 'clf_lr__fit_intercept',
 'clf_lr__intercept_scaling',
 'clf_lr__max_iter',
 'clf_lr__multi_class',
 'clf_lr__n_jobs',
 'clf_lr__penalty',
 'clf_lr__random_state',
 'clf_lr__solver',
 'clf_lr__tol',
 'clf_lr__verbose',
 'clf_lr__warm_start',
 'steps',
 'vectorizer',
 'vectorizer__analyzer',
 'vectorizer__binary',
 'vectorizer__decode_error',
 'vectorizer__dtype',
 'vectorizer__encoding',
 'vectorizer__input',
 'vectorizer__lowercase',
 'vectorizer__max_df',
 'vectorizer__max_features',
 'vectorizer__min_df',
 'vectorizer__ngram_range',
 'vectorizer__preprocessor',
 'vectorizer__stop_words',
 'vectorizer__strip_accents',
 'vectorizer__token_pattern',
 'vectorizer__tokenizer',
 'vectorizer__vocabulary']

In [39]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import StratifiedKFold
parameters = { 'vectorizer__max_df':[0.001, 0.01, 0.1, 0.5, 1]
             }
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

In [ ]:
#use gridsearch over specified parameter values for an estimator
grid_search = GridSearchCV(pipeline_lr, parameters, n_jobs=-1, verbose=1, scoring="roc_auc", cv=3)
grid_result = grid_search.fit(X=train.reviewText, y=train.overall)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


In [15]:
test["overall"] = pipeline_lr.predict_proba(test["reviewText"])[:,1]

In [16]:
test[["index","overall"]].head()

,index,overall
0,0,0.783648
1,1,0.492577
2,2,0.002617
3,3,0.969034
4,4,0.136881


In [17]:
test[["index","overall"]].to_csv("baseline_submission.csv", index=False)